In [6]:
pip install --upgrade google-cloud-vision


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
from google.cloud import vision
import os
import io
import json

# 서비스 계정 키
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gcloud service key'

client = vision.ImageAnnotatorClient()

file_path = 'img_path'

# 이미지 파일 바이트로 읽기
with io.open(file_path, 'rb') as image_file:
    content = image_file.read()

# Vision API 기능 사용 (class instance 생성)    
image = vision.Image(content=content)

# 문서 텍스트 감지 + 추출
response = client.document_text_detection(image=image)
# texts = response.text_annotations
texts = response.text_annotations[0].description if response.text_annotations else ""
print(texts)
# JSON 파일 형식으로 변환
#result = {'texts': [text.description for text in texts]}

# JSON 결과를 표준 출력으로 출력
#print(json.dumps(result, indent=4))

California Import Parts Ltd. / www.cip1.com
Your #1 source for restoration and performance parts for air-cooled Volkswagens
1-604-881-7858
1-877-811-5111
1-604-881-7858
Reason for shipping
Goods Sold
Vintage Collectable Auto Parts
Tax# 98-0366890
Local
Fax
Tech Line
Shipping Information:
All prices in USD
CIP1 is Importer of Record
This order qualifies under Section 321
Order Toll Free
1-800-313-3811
Number of Pieces ------>
This order qualifies under Section 321
In the USA:
PMB #108
1124 Fir Ave
Blaine WA, 98230
In Canada:
113 19168 39th Ave
Surrey BC,V3Z 0Y6
NET TOTAL $24.96
SUB TOTAL $24.96
FREIGHT
TOTAL
$9.99
$34.95


In [8]:
pip install openai


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from openai import OpenAI
import re

api_key = ''

client = OpenAI(api_key=api_key)

prompt = f"""Extract important information from the following document and organize the data into the specified JSON format:
Document:
{texts}

Return the information in JSON format with the following structure: owner, customer, po_no, po_date, company_address, customer_address, shipping_address_name, grand_total, items (including item_name, description, qty, rate, amount, income_account), and taxes (including charge_type, account_head, description, tax_amount).
"""

# OpenAI API 써서 필요한 정보 고르고, 기존에 정한 json 형식이랑 매핑
response = client.completions.create(
    model="gpt-3.5-turbo-instruct",  
    prompt=prompt,
    max_tokens=1000,
    temperature=0.3
)

# 얻은 텍스트
response_text = response.choices[0].text.strip()

# 정보를 추출하고, 일치하는 정보가 없으면 "" 넣음
def extract_with_default(pattern, text, default=""):
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return default

try:
    # 동적으로 값 할당 (이래야 openAI가 매핑한 ... 결과가 동적으로 들어감)
    structured_response = json.loads(response.choices[0].text.strip())
    data_to_json = {"data": structured_response}
except json.JSONDecodeError:
    print("Failed to decode the response into JSON")
    data_to_json = {}

# 결과 출력
print(json.dumps(data_to_json, indent=4))



{
    "data": {
        "owner": "California Import Parts Ltd.",
        "customer": "www.cip1.com",
        "po_no": "",
        "po_date": "",
        "company_address": {
            "phone": "1-604-881-7858",
            "fax": "1-604-881-7858",
            "tax_number": "Tax# 98-0366890",
            "local": "Local",
            "tech_line": "Tech Line",
            "website": "www.cip1.com"
        },
        "customer_address": {
            "phone": "1-877-811-5111",
            "fax": "",
            "tax_number": "",
            "local": "",
            "tech_line": "",
            "website": ""
        },
        "shipping_address_name": "Reason for shipping",
        "grand_total": "$34.95",
        "items": [
            {
                "item_name": "Goods Sold",
                "description": "Vintage Collectable Auto Parts",
                "qty": "1",
                "rate": "$24.96",
                "amount": "$24.96",
                "income_account": ""
          